In [ ]:
import sqlalchemy as sqla
import pandas as pd
import getpass

# some variables for tables
_succ = True

while _succ:
    try:
        _dbms = 'postgresql'
        _server = input('Please provide server address (default 127.0.0.1):') or '127.0.0.1'
        _db = input('Please provide database name')
        _schema = input('Please provide schema name (default public):') or 'public'
        _user = input('Please provide user name for database')
        _passw = getpass.getpass('Please provide  password for database')
        _url = f'{_dbms}://{_user}:{_passw}@{_server}/{_db}'
        _eng = sqla.create_engine(_url)
        with _eng.connect() as conn:
            pass
    except Exception as e:
        print('There was error')
        print(e)
        print(f'{"try again":=^86}')
    else:
        _succ = False
        print('Engine created succesfuly')
        

def query(_stat, _eng=_eng):
    with _eng.connect() as _conn:
        _results = _conn.execute(sqla.text(_stat)).fetchall()
    return _results


def crt_query(_stat, _eng=_eng, ):
    with _eng.connect() as _conn:
        _conn.execute(sqla.text(_stat))
        _conn.commit()
    return True


def insert_query(_schema, _table_name, _data, _eng=_eng):
    """Insert data query, _stat should be like "INSERT INTO table (col1, col2) VALUES (:col1, :col2)"
    data should be like [{"col1": val1, "col2": val2}, {"col1": val3, "col2": val4}] """

    _data = [ {k: (None if pd.isna(v) else v) for k, v in row.items()} for row in _data ]    
    
    # first check if columns from data are subset of columns from db
    columns_from_db = set(pd_query(f"SELECT * FROM {_schema}.{_table_name} LIMIT 0").columns)
    columns_from_data = set(key for row in _data for key in row.keys())
    # raise Valueerror if not subset
    if not columns_from_data.issubset(columns_from_db):
        message = 'Mismatch in column names:\n'
        message += f'columns_from_db:\t{sorted(columns_from_db)}\n'
        message += f'columns_from_data:\t{sorted(columns_from_data)}'
        raise ValueError(message)

    # next create insert statement
    cols_str = ", ".join(_data[0].keys())
    placeholders = ", ".join(f":{column}" for column in _data[0].keys())
    stat = f"""
            INSERT INTO {_schema}.{_table_name} ({cols_str}) VALUES ({placeholders})
            """.strip()
    #connect to db, execute and commit
    with _eng.connect() as _conn:
        _conn.execute(sqla.text(stat), _data)
        _conn.commit()
    return True


def pd_query(_stat, _eng=_eng):
    return pd.read_sql_query(sqla.text(_stat), _eng)

# Create database
* Create database and grant all privileges to user with grant options
    * May not work sqlalchemy method so create from pgAdmin or psql
* Retrun to first cell and rerun it to update _url variable(s)

In [ ]:
stat = f"""
CREATE DATABASE {_db}

ALTER DATABASE {_db} OWNER TO {_user};

ALTER DEFAULT PRIVILEGES FOR ROLE postgres
GRANT EXECUTE ON FUNCTIONS TO {_user} WITH GRANT OPTION;

GRANT ALL ON DATABASE {_db} TO {_user} WITH GRANT OPTION;

ALTER DEFAULT PRIVILEGES FOR ROLE postgres
GRANT ALL ON TABLES TO {_user} WITH GRANT OPTION;

ALTER DEFAULT PRIVILEGES FOR ROLE postgres
GRANT ALL ON SEQUENCES TO {_user} WITH GRANT OPTION;

ALTER DEFAULT PRIVILEGES FOR ROLE postgres
GRANT USAGE ON TYPES TO {_user} WITH GRANT OPTION;
"""
crt_query(stat)

# Create (modify) Schema
reaname schema and correct privilegies

In [ ]:
stat = f"""
ALTER SCHEMA {_schema} OWNER TO {_user};

COMMENT ON SCHEMA {_schema} IS 'default schema';

REVOKE ALL ON SCHEMA {_schema} FROM PUBLIC;
GRANT ALL ON SCHEMA {_schema} TO {_user} WITH GRANT OPTION;

ALTER DEFAULT PRIVILEGES FOR ROLE {_user} IN SCHEMA {_schema}
GRANT ALL ON TABLES TO {_user} WITH GRANT OPTION;

ALTER DEFAULT PRIVILEGES FOR ROLE {_user} IN SCHEMA {_schema}
GRANT ALL ON SEQUENCES TO {_user} WITH GRANT OPTION;

ALTER DEFAULT PRIVILEGES FOR ROLE {_user} IN SCHEMA {_schema}
GRANT EXECUTE ON FUNCTIONS TO {_user} WITH GRANT OPTION;

ALTER DEFAULT PRIVILEGES FOR ROLE {_user} IN SCHEMA {_schema}
GRANT USAGE ON TYPES TO {_user} WITH GRANT OPTION;
"""
crt_query(stat)

## lookup Tables 
* units
* stone_names
* stone_qualities
* genders
* model_categories
* masters
* transaction_types
* item_types
* assets
* product_location

In [ ]:
# function to create lookup tables and poulate them
def create_lookup_tables(lookup_table):
    """Creates lookup table based on arguments name and dtype"""
    global _schema
    global _user
    name = lookup_table['tb_name']
    dtype = lookup_table['dtype']
    data = lookup_table['data']
    stat = f"""
    CREATE TABLE IF NOT EXISTS {_schema}.{name}
    (
        label {dtype} NOT NULL,
        note text, 
        PRIMARY KEY (label)
    )
    TABLESPACE pg_default;
    COMMENT ON TABLE {_schema}.{name} IS 'lookup values';
    """
    # create table and print True if successful
    print(f'table {name} created: {crt_query(stat)}', end="; ")
    # insert data and print true if succesful
    print(f'data inserted {insert_query(_schema, name, data)}')

In [ ]:
lookup_tables = [
            {'tb_name':'units', 
                 'data':[{'label':'გრამი'}, {'label':'კარატი'}, {'label':'ცალი'}, 
                         {'label':'მილიმეტრი'}, {'label':'კილოგრამი'}, {'label':'ლარი'}, 
                         {'label':'NA'}, {'label':'ნაჭერი'}, ], 
                 'dtype':'text'},
            {'tb_name':'stone_names', 
                 'data':[{'label':'ბრილიანტი'}, {'label':'ცირკონი'}, {'label':'ბაგეტი'}], 
                 'dtype':'text'},
            {'tb_name':'stone_qualities', 
                 'data':[{'label':'დაბალი'}, {'label':'საშუალო'}, {'label':'მაღალი'}], 
                 'dtype':'text'},    
            {'tb_name':'genders', 
                 'data':[{'label':'ქალი'}, {'label':'კაცი'}, ], 
                 'dtype':'text'}, 
            {'tb_name':'model_categories', 
                 'data':[{'label':'ბეჭედი'}, {'label':'საყურე'}, ], 
                 'dtype':'text'},
            {'tb_name':'masters', 
                 'data':[{'label':'კოკელაძე გიორგი'}, ], 
                 'dtype':'text'},    
            {'tb_name':'transaction_types', 
                 'data':[{'label':'თავდაპირველი მარაგი'}, {'label':'შესყიდვა'}, {'label':'გადაყვანა'},
                         {'label':'ჩამოსხმა'}, {'label':'დამუშავება'}, {'label':'გაყიდვა'}, ],
                 'dtype':'text'},
            {'tb_name':'item_types', 
                 'data':[{'label':'ქვა'}, {'label':'მეტალი'}, {'label':'სხვა მატერიალი'}, 
                         {'label':'მომსახურება'}, {'label':'შემოსავალი'}, ], 
                 'dtype':'text'},
            {'tb_name':'assets', 
                 'data':[{'label':'სანთელი'}, {'label':'ფული'}, {'label':'საჭმელი'}, ], 
                 'dtype':'text'}, 
            {'tb_name':'product_location', 
                 'data':[{'label':'სეიფი'}, {'label':'დახლი'}, {'label':'გაყიდული'}, ], 
                 'dtype':'text'},     
                ]

for table in lookup_tables:
    create_lookup_tables(table)

## Table customers

In [ ]:
name = 'customers'
comment = f'customer names and info'
prim_key = ['full_name']

stat = f"""
CREATE TABLE IF NOT EXISTS {_schema}.{name}
(
    {prim_key[0]} text NOT NULL,
    phone text,
    table_number text,
    id text,
    address text,
    note text, 
    PRIMARY KEY ({', '.join(prim_key)})
)
TABLESPACE pg_default;
COMMENT ON TABLE {_schema}.{name} IS '{comment}';
"""
# prepare data to load to table
data = [{'full_name':'NA'}, {'full_name':'მარეხი'}, {'full_name':'facebook'}, ]
# create table
print(crt_query(stat))
# insert data
print(insert_query(_schema, name, data))

## Table metals

In [ ]:
# create metals table and populate it from dictionary
name = 'metals'
comment = f'metals with purity (sinji) and their characteristics if any'
prim_key = ['metal_full_name']

stat = f"""
CREATE TABLE IF NOT EXISTS {_schema}.{name}
(
    metal_name text NOT NULL,
    sinji integer NOT NULL,
    {prim_key[0]} text NOT NULL 
        GENERATED ALWAYS AS ( metal_name || '-' || sinji::text ) STORED,
    note text,
    django_id Serial NOT NULL,
    PRIMARY KEY ({', '.join(prim_key)})
)
TABLESPACE pg_default;
COMMENT ON TABLE {_schema}.{name} IS '{comment}';
"""
# prepare data to load to table
data = [ {'metal_name':'ოქრო', 'sinji':995}, {'metal_name':'ოქრო', 'sinji':585}, ]
# create table
print(crt_query(stat))
# insert data
print(insert_query(_schema, name, data))

## Table Stones

In [ ]:
# create stones table and populate it from excel spreadhshit
name = 'stones'
comment = f'stones and their characteristics'
prim_key = ['stone_full_name']

stat = f"""
CREATE TABLE IF NOT EXISTS {_schema}.{name}
(
    stone_name text NOT NULL,
    size text NOT NULL DEFAULT 'NA',
    size_unit text DEFAULT 'მილიმეტრი',
    weight numeric(10, 4) DEFAULT 0,
    weight_unit text DEFAULT 'კარატი',
    {prim_key[0]} text NOT NULL 
        GENERATED ALWAYS AS ( stone_name || '-' || size ) STORED,
    note text, 
    django_id Serial NOT NULL,
    PRIMARY KEY ({', '.join(prim_key)}),
    FOREIGN KEY (stone_name) REFERENCES {_schema}.stone_names (label) MATCH SIMPLE ON UPDATE CASCADE,
    FOREIGN KEY (size_unit) REFERENCES {_schema}.units (label) MATCH SIMPLE ON UPDATE CASCADE,
    FOREIGN KEY (weight_unit) REFERENCES {_schema}.units (label) MATCH SIMPLE ON UPDATE CASCADE
)
TABLESPACE pg_default;
COMMENT ON TABLE {_schema}.{name} IS '{comment}';
"""

# create table
print(crt_query(stat))

## Table catalog

In [ ]:
name = 'catalog'
comment = f'catalog of products'
prim_key = ['model_id']

stat = f"""
CREATE TABLE IF NOT EXISTS {_schema}.{name}
(
    {prim_key[0]} text NOT NULL,
    creation_date date NOT NULL DEFAULT CURRENT_DATE,
    pieces integer,
    model_name text,
    model_category text,
    gender text,
    image_location text,
    note text, 
    PRIMARY KEY ({', '.join(prim_key)}),
    FOREIGN KEY (gender) REFERENCES {_schema}.genders (label) MATCH SIMPLE ON UPDATE CASCADE,
    FOREIGN KEY (model_category) REFERENCES {_schema}.model_categories (label) MATCH SIMPLE ON UPDATE CASCADE
)
TABLESPACE pg_default;
COMMENT ON TABLE {_schema}.{name} IS '{comment}';
"""

# create table
print(crt_query(stat))

## Table catalog_stones

In [ ]:
name = 'catalog_stones'
comment = f'stones of models in catalog, only stones, but can be used for other if needed'
prim_key = ['model_id', 'stone_full_name']

stat = f"""
CREATE TABLE IF NOT EXISTS {_schema}.{name}
(
    {prim_key[0]} text NOT NULL,
    {prim_key[1]} text NOT NULL,
    quantity integer NOT NULL,
    note text, 
    django_id Serial NOT NULL,
    PRIMARY KEY ({', '.join(prim_key)}),
    FOREIGN KEY ({prim_key[0]}) REFERENCES {_schema}.catalog ({prim_key[0]})
        MATCH SIMPLE ON UPDATE CASCADE ON DELETE CASCADE,
    FOREIGN KEY ({prim_key[1]}) REFERENCES {_schema}.stones  ({prim_key[1]}) 
        MATCH SIMPLE ON UPDATE CASCADE
)
TABLESPACE pg_default;
COMMENT ON TABLE {_schema}.{name} IS '{comment}';
"""

# create table
print(crt_query(stat))

## Table lots

In [ ]:
name = 'lots'
comment = f'lots info'
prim_key = ['lot_id']

stat = f"""
CREATE TABLE IF NOT EXISTS {_schema}.{name}
(
    {prim_key[0]} serial NOT NULL,
    lot_date date NOT NULL DEFAULT CURRENT_DATE,
    metal_full_name text, 
    master text,
    cost_grinding numeric(10, 2) NOT NULL,
    cost_manufacturing_stone numeric(10, 2) NOT NULL,
    cost_polishing numeric(10, 2) NOT NULL,
    cost_plating numeric(10, 2) NOT NULL,
    cost_sinji numeric(10, 2) NOT NULL,
    margin_stones numeric(10, 2) NOT NULL,
    price_gram_gold numeric(10, 2) NOT NULL,
    note text, 
    PRIMARY KEY ({', '.join(prim_key)}),
    FOREIGN KEY (metal_full_name) REFERENCES {_schema}.metals (metal_full_name) 
        MATCH FULL ON UPDATE CASCADE,
    FOREIGN KEY (master) REFERENCES {_schema}.masters (label) 
        MATCH FULL ON UPDATE CASCADE        
)
TABLESPACE pg_default;
COMMENT ON TABLE {_schema}.{name} IS '{comment}';
"""

# create table
print(crt_query(stat))

## TABLE transactions

In [ ]:
name = 'transactions'
comment = f'all transactions related to material purchases, service purchases, sallary payments ...'
prim_key = ['tmstmp']

stat = f"""
CREATE TABLE IF NOT EXISTS {_schema}.{name}
(
    {prim_key[0]} text NOT NULL DEFAULT to_char(CURRENT_TIMESTAMP, 'YY-MM-DD-HH24-MI-SS-US'),
    item text NOT NULL,
    item_type text NOT NULL,
    transaction_type text NOT NULL,
    description text,
    lot_id integer,
    customer text,
    transaction_quantity numeric(10, 4) NOT NULL,
    transaction_quantity_unit text NOT NULL,
    pieces numeric(10, 4),
    stone_quality text ,
    cost_unit numeric(10, 2) NOT NULL,
    total_cost numeric(10, 2) GENERATED ALWAYS AS ( transaction_quantity * cost_unit ) STORED,
    cost_piece numeric(10, 2) GENERATED ALWAYS AS ( (transaction_quantity * cost_unit) / pieces ) STORED,
    image_location text,
    note text, 
    django_id Serial NOT NULL,
    PRIMARY KEY ({', '.join(prim_key)}),
    FOREIGN KEY (item_type) REFERENCES {_schema}.item_types (label) MATCH FULL ON UPDATE CASCADE,
    FOREIGN KEY (transaction_type) REFERENCES {_schema}.transaction_types (label) MATCH SIMPLE ON UPDATE CASCADE,
    FOREIGN KEY (lot_id) REFERENCES {_schema}.lots (lot_id) MATCH SIMPLE ON UPDATE CASCADE,
    FOREIGN KEY (transaction_quantity_unit) REFERENCES {_schema}.units (label) MATCH SIMPLE ON UPDATE CASCADE,
    FOREIGN KEY (stone_quality) REFERENCES {_schema}.stone_qualities (label) MATCH SIMPLE ON UPDATE CASCADE,
    FOREIGN KEY (customer) REFERENCES {_schema}.customers (full_name) MATCH SIMPLE ON UPDATE CASCADE
)
TABLESPACE pg_default;
COMMENT ON TABLE {_schema}.{name}
    IS '{comment}';
"""

data
# create table
print(crt_query(stat))

## Table lot_models

In [ ]:
name = 'lot_models'
comment = f'models added to lot for production'
prim_key = ['lot_id', 'model_id', 'tmstmp']

stat = f"""
CREATE TABLE IF NOT EXISTS {_schema}.{name}
(
    {prim_key[0]} integer NOT NULL,
    {prim_key[1]} text NOT NULL,
    {prim_key[2]} text NOT NULL DEFAULT to_char(CURRENT_TIMESTAMP, 'YY-MM-DD-HH24-MI-SS'),
    weight numeric(10, 2),
    customer text NOT NULL DEFAULT 'NA',
    sale_date date,
    location text NOT NULL DEFAULT 'სეიფი',
    cost_gram_gold numeric(10, 2),    
    price_gram_gold numeric(10, 2),
    model_gold_cost numeric(10, 2) 
        GENERATED ALWAYS AS (cost_gram_gold  * weight) STORED,
    model_gold_price numeric(10, 2) 
        GENERATED ALWAYS AS (price_gram_gold * weight) STORED,
    note text, 
    PRIMARY KEY ({', '.join(prim_key)}),
    FOREIGN KEY ({prim_key[0]}) REFERENCES {_schema}.lots ({prim_key[0]}) 
        MATCH SIMPLE ON UPDATE CASCADE ON DELETE CASCADE,
    FOREIGN KEY ({prim_key[1]}) REFERENCES {_schema}.catalog ({prim_key[1]}) 
        MATCH SIMPLE ON UPDATE CASCADE,
    FOREIGN KEY (customer) REFERENCES {_schema}.customers (full_name) 
        MATCH SIMPLE ON UPDATE CASCADE,
    FOREIGN KEY (location) REFERENCES {_schema}.product_location (label) 
        MATCH SIMPLE ON UPDATE CASCADE
)
TABLESPACE pg_default;
COMMENT ON TABLE {_schema}.{name} IS '{comment}';
"""

# create table
print(crt_query(stat))

## Table  lot_model_stones

In [ ]:
name = 'lot_model_stones'
comment = f'stones of model under lot in production'
prim_key = ['lot_id', 'model_id', 'tmstmp', 'stone_full_name']

stat = f"""
CREATE TABLE IF NOT EXISTS {_schema}.{name}
(
    {prim_key[0]} integer NOT NULL,
    {prim_key[1]} text NOT NULL,
    {prim_key[2]} text NOT NULL DEFAULT to_char(CURRENT_TIMESTAMP, 'YY-MM-DD-HH24-MI-SS'),
    {prim_key[3]} text NOT NULL,
    quantity integer NOT NULL,
    weight numeric(10, 4),
    weight_unit text,
    cost_piece numeric(10, 2),
    cost_manufacturing_stone numeric(10, 2),
    margin_stones numeric(10, 2),
    installed boolean NOT NULL DEFAULT false,
    total_weight numeric(10, 4) 
        GENERATED ALWAYS AS (quantity * weight) STORED,
    total_cost_piece numeric(10, 2) 
        GENERATED ALWAYS AS (quantity * cost_piece) STORED,
    total_cost_manufacturing_stone numeric(10, 2) 
        GENERATED ALWAYS AS (quantity * cost_manufacturing_stone) STORED,
    total_cost numeric(10, 2) 
        GENERATED ALWAYS AS (quantity * (cost_piece + cost_manufacturing_stone)) STORED,
    total_margin_stones numeric(10, 2) 
        GENERATED ALWAYS AS (quantity * margin_stones) STORED,
    total_price numeric(10, 2) 
        GENERATED ALWAYS AS (quantity * (cost_piece + cost_manufacturing_stone + margin_stones)) STORED,
    note text,
    django_id Serial NOT NULL,
    PRIMARY KEY ({', '.join(prim_key)}),
    FOREIGN KEY ({', '.join(prim_key[0:3])}) REFERENCES {_schema}.lot_models ({', '.join(prim_key[0:3])})
        MATCH FULL ON UPDATE CASCADE ON DELETE CASCADE,
    FOREIGN KEY ({prim_key[3]}) REFERENCES {_schema}.stones  ({prim_key[3]}) 
        MATCH FULL ON UPDATE CASCADE
)
TABLESPACE pg_default;
COMMENT ON TABLE {_schema}.{name} IS '{comment}';
"""

# create table
print(crt_query(stat))

## trigger add_model_2_lot_models_trigger

In [ ]:
# triger function that will insert records from catalog_stones and stones to lot_model_stones
name = 'add_model_2_lot_models_trigger'
comment = f'trigger function that adds stones to model in lot'

stat = f"""
CREATE OR REPLACE FUNCTION {_schema}.{name}()
    RETURNS TRIGGER
    LANGUAGE plpgsql
AS $BODY$
BEGIN
    IF (TG_OP = 'INSERT') THEN
     
        INSERT INTO {_schema}.lot_model_stones 
            (lot_id, model_id, tmstmp, stone_full_name, quantity, 
                cost_piece, cost_manufacturing_stone, margin_stones)
            SELECT NEW.lot_id, NEW.model_id, NEW.tmstmp, cs.stone_full_name, cs.quantity, 
                t.cost_piece, l.cost_manufacturing_stone, l.margin_stones
            FROM {_schema}.catalog_stones AS cs
                LEFT JOIN {_schema}.stones AS s ON cs.stone_full_name = s.stone_full_name
                LEFT JOIN {_schema}.lots AS l ON l.lot_id = NEW.lot_id
                LEFT JOIN (
                    SELECT item, MAX( cost_piece ) AS cost_piece
            		FROM {_schema}.transactions
            		WHERE tmstmp > to_char(CURRENT_TIMESTAMP - '6 months'::INTERVAL, 'YY-MM-DD-HH24-MI-SS')
            		GROUP BY item) AS t ON cs.stone_full_name = t.item
            WHERE model_id = NEW.model_id;

        IF (NEW.price_gram_gold IS NULL) THEN

            UPDATE {_schema}.lot_models 
                SET price_gram_gold = (SELECT price_gram_gold FROM {_schema}.lots WHERE lot_id = NEW.lot_id)
            WHERE lot_id = NEW.lot_id AND model_id = NEW.model_id AND tmstmp = NEW.tmstmp;

        END IF;
        
    END IF;
    RETURN NULL;
END;
$BODY$;
COMMENT ON FUNCTION {_schema}.{name}() IS '{comment}';
"""
print(crt_query(stat))

## Trigger on model add
* on INSERT in lot_models
* calls add_model_2_lot_models_trigger()

In [ ]:
# Create insert trigger using above trigger function
name = 'add_model_2_lot_models_trigger'
triger_table = 'lot_models'
comment = f'trigger when adding new record (model) to lot'

stat = f"""
CREATE OR REPLACE TRIGGER {name}
    AFTER INSERT
    ON {_schema}.{triger_table}
    FOR EACH ROW
    EXECUTE FUNCTION {_schema}.{name}();
    
--comment
COMMENT ON TRIGGER {name} ON {_schema}.{triger_table} IS '{comment}';
"""

print(crt_query(stat))

## trigger insert_update_stone_on_lot_model_stones_trigger

In [ ]:
# triger function that will update weight, weight_unit in lot_model_stones
name = 'insert_update_stone_on_lot_model_stones_trigger'
comment = f'trigger function to update weight, weight_unit and cost_piece in lot_model_stones'

stat = f"""
CREATE OR REPLACE FUNCTION {_schema}.{name}()
    RETURNS TRIGGER
    LANGUAGE plpgsql
AS $BODY$
BEGIN
    IF TG_OP IN ('UPDATE', 'INSERT') THEN

        UPDATE {_schema}.lot_model_stones 
            SET weight = (SELECT weight FROM {_schema}.stones WHERE stone_full_name = NEW.stone_full_name), 
                weight_unit = (SELECT weight_unit FROM {_schema}.stones 
                    WHERE stone_full_name = NEW.stone_full_name)
        WHERE lot_id = NEW.lot_id AND model_id = NEW.model_id 
            AND tmstmp = NEW.tmstmp
            AND stone_full_name = NEW.stone_full_name;
        
    END IF;
    RETURN NULL;
END;
$BODY$;
COMMENT ON FUNCTION {_schema}.{name}() IS '{comment}';
"""
print(crt_query(stat))

## Trigger on stone update
* after insert or update of stone_full_name in lot_model_stones
* calls insert_update_stone_on_lot_model_stones_trigger()

In [ ]:
# Create insert and update trigger using above trigger function
name = 'insert_update_stone_on_lot_model_stones_trigger'
triger_table = 'lot_model_stones'
comment = f'trigger when updating stone in lot model'

stat = f"""
CREATE OR REPLACE TRIGGER {name}
    AFTER INSERT OR UPDATE OF stone_full_name
    ON {_schema}.{triger_table}
    FOR EACH ROW
    EXECUTE FUNCTION {_schema}.{name}();
    
--comment
COMMENT ON TRIGGER {name} ON {_schema}.{triger_table} IS '{comment}';
"""
print(crt_query(stat))

## VIEW  records_for_summary

In [ ]:
name = 'records_for_summary'
comment = 'transaction records for summary page'

stat = f"""
CREATE OR REPLACE VIEW {_schema}.{name}
AS
SELECT lot_id, description, tmstmp, transaction_type, item,  item_type, 
    CONCAT(transaction_quantity::NUMERIC(10,2), ' ', transaction_quantity_unit) AS transaction_quantity, 
    cost_unit, total_cost, 'act' AS act
FROM {_schema}.transactions;

COMMENT ON VIEW {_schema}.{name}
    IS '{comment}';
"""
# create view
print(crt_query(stat))

## VIEW  total_metals

In [ ]:
name = 'total_metals_per_metal'
comment = 'Metals grouped by name and unit cost showing total quantity, unit cost and total cost'

stat = f"""
CREATE OR REPLACE VIEW {_schema}.{name}
AS
SELECT item, 
    SUM(transaction_quantity) AS total_quantity, 
    cost_unit, 
    SUM(total_cost) AS total_cost
FROM {_schema}.transactions 
WHERE item_type = 'მეტალი' AND item NOT LIKE '%დანაკარგი%'
GROUP BY item, transaction_quantity_unit, cost_unit 
HAVING SUM(transaction_quantity) != 0 AND SUM(total_cost) != 0
ORDER BY item ;

COMMENT ON VIEW {_schema}.{name}
    IS '{comment}';
"""
# create view
print(crt_query(stat))

## VIEW  total_other_materials

In [ ]:
name = 'total_other_materials'
comment = 'Other_materials grouped by name and unit cost showing total quantity, unit cost and total cost'

stat = f"""
CREATE OR REPLACE VIEW {_schema}.{name}
AS
WITH cash AS(
	SELECT item, 
	    '-' AS total_quantity, 
	    0 AS cost_unit, 
	    SUM(total_cost) AS total_cost
	FROM {_schema}.transactions 
	WHERE item = 'ფული'
	GROUP BY item
	), materials AS(
	SELECT item, 
	    CONCAT(SUM(transaction_quantity), ' ', transaction_quantity_unit) AS total_quantity, 
	    cost_unit, 
	    SUM(total_cost) AS total_cost
	FROM {_schema}.transactions 
	WHERE item_type = 'სხვა მატერიალი'
	GROUP BY item, transaction_quantity_unit, cost_unit 
    HAVING SUM(transaction_quantity) != 0 AND SUM(total_cost) != 0
    ORDER BY item
	)
SELECT *
FROM cash
UNION
SELECT *
FROM materials;

COMMENT ON VIEW {_schema}.{name}
    IS '{comment}';
"""
# create view
print(crt_query(stat))

## VIEW  total_sinji


In [ ]:
name = 'total_sinji_per_date'
comment = 'Total metal quantities taken and returned for sinji'

stat = f"""
CREATE OR REPLACE VIEW {_schema}.{name}
AS
WITH uno AS(
    SELECT to_date(tmstmp, 'YY-MM-DD') AS transaction_date, 
        ABS(SUM(transaction_quantity)) AS total_weight_in,
        ABS(SUM(total_cost)) AS total_cost_in
    FROM {_schema}.transactions
    WHERE transaction_type = 'გადაყვანა' AND item_type = 'მეტალი' AND transaction_quantity < 0
    GROUP BY to_date(tmstmp, 'YY-MM-DD')
), due AS(
    SELECT to_date(tmstmp, 'YY-MM-DD') AS transaction_date, 
        ABS(SUM(transaction_quantity)) AS total_weight_out
    FROM {_schema}.transactions
    WHERE transaction_type = 'გადაყვანა' AND item_type = 'მეტალი' AND transaction_quantity >= 0
    GROUP BY to_date(tmstmp, 'YY-MM-DD')
)
SELECT uno.transaction_date, 
    COALESCE(uno.total_weight_in, 0) AS total_weight_in, 
    COALESCE(due.total_weight_out, 0) AS total_weight_out, 
    COALESCE(uno.total_weight_in - due.total_weight_out, 0)  AS total_weight_lost,
    COALESCE(uno.total_cost_in, 0) AS total_cost_in,
    COALESCE(ROUND(uno.total_cost_in / due.total_weight_out, 2), 0) AS cost_per_gram
FROM uno
	LEFT JOIN due ON uno.transaction_date = due.transaction_date
ORDER BY uno.transaction_date DESC;

COMMENT ON VIEW {_schema}.{name}
    IS '{comment}';
"""
# create view
print(crt_query(stat))

## VIEW  total_cast_per_lot

In [ ]:
name = 'total_cast_per_lot'
comment = 'Total Weight of casted gold by lot'

stat = f"""
CREATE OR REPLACE VIEW {_schema}.{name}
AS
WITH uno AS(
    SELECT lot_id, 
        ABS(SUM(transaction_quantity)) AS total_weight_in
    FROM {_schema}.transactions
    WHERE transaction_type = 'ჩამოსხმა' AND item_type = 'მეტალი' AND transaction_quantity < 0
    GROUP BY lot_id
), due AS(
    SELECT lot_id, 
        ABS(SUM(transaction_quantity)) AS total_weight_out
    FROM {_schema}.transactions
    WHERE transaction_type = 'ჩამოსხმა' AND item_type = 'მეტალი'
    GROUP BY lot_id
), tres AS(
    SELECT lot_id, 
        SUM(transaction_quantity) AS total_weight_lost,
        SUM(total_cost) AS total_cost_lost
    FROM {_schema}.transactions
    WHERE transaction_type = 'ჩამოსხმა' AND item ILIKE ('%დანაკარგ%')
    GROUP BY lot_id
), quatro AS(
    SELECT lot_id, 
        SUM(ABS(total_cost)) AS total_cost
    FROM {_schema}.transactions
    WHERE transaction_type = 'ჩამოსხმა' AND item_type != 'მეტალი'
    GROUP BY lot_id
), cinco AS(
    SELECT lot_id, 
        SUM(ABS(total_cost)) AS total_salary
    FROM {_schema}.transactions
    WHERE transaction_type = 'ჩამოსხმა' AND item = 'ფული'
    GROUP BY lot_id
)
SELECT lots.lot_id, 
    uno.total_weight_in,
    due.total_weight_out,
    COALESCE(tres.total_weight_lost, 0) AS total_weight_lost,
    COALESCE(tres.total_cost_lost, 0) AS total_cost_lost,
    COALESCE(quatro.total_cost, 0) AS total_cost,
    COALESCE(cinco.total_salary, 0) AS total_salary
FROM {_schema}.lots
	LEFT JOIN uno ON lots.lot_id = uno.lot_id
	LEFT JOIN due ON lots.lot_id = due.lot_id
	LEFT JOIN tres ON lots.lot_id = tres.lot_id
    LEFT JOIN quatro ON lots.lot_id = quatro.lot_id
    LEFT JOIN cinco ON lots.lot_id = cinco.lot_id
ORDER BY lots.lot_id DESC;

COMMENT ON VIEW {_schema}.{name}
    IS '{comment}';
"""
# create view
print(crt_query(stat))

## VIEW  total_processing_per_lot

In [ ]:
name = 'total_processing_per_lot'
comment = 'Total Weight of peocessed gold by lot'

stat = f"""
CREATE OR REPLACE VIEW {_schema}.{name}
AS
WITH cero AS(
    SELECT lot_id,
        ABS(SUM(transaction_quantity)) as aditional_metali_in
    FROM transactions
    WHERE transaction_quantity <= 0 AND transaction_type = 'დამუშავება' AND item_type = 'მეტალი'
    GROUP BY lot_id
), uno AS(
    SELECT lot_id, 
        ABS(SUM(transaction_quantity)) AS total_weight_in
    FROM {_schema}.transactions
    WHERE transaction_type = 'ჩამოსხმა' AND item_type = 'მეტალი'
    GROUP BY lot_id
), due AS(
    SELECT lot_id, 
        ABS(SUM(transaction_quantity)) AS total_weight_out,
        ABS(SUM(total_cost)) AS total_cost_out
    FROM {_schema}.transactions
    WHERE item_type = 'მეტალი'
    GROUP BY lot_id
), tres AS(
    SELECT lot_id, 
        SUM(transaction_quantity) AS total_weight_lost,
        SUM(total_cost) AS total_cost_lost
    FROM {_schema}.transactions
    WHERE transaction_type = 'დამუშავება' AND item ILIKE ('%დანაკარგ%')
    GROUP BY lot_id
), quatro AS(
    SELECT lot_id, 
        SUM(ABS(total_cost)) AS total_cost
    FROM {_schema}.transactions
    WHERE transaction_type = 'დამუშავება' AND item_type != 'მეტალი'
    GROUP BY lot_id
), cinco AS(
    SELECT lot_id, 
        SUM(ABS(total_cost)) AS total_salary
    FROM {_schema}.transactions
    WHERE transaction_type = 'დამუშავება' AND item = 'ფული'
    GROUP BY lot_id
)
SELECT lots.lot_id, 
    COALESCE(uno.total_weight_in + cero.aditional_metali_in, 0) AS total_weight_in,
    due.total_weight_out, 
    due.total_cost_out,
    COALESCE(tres.total_weight_lost, 0) AS total_weight_lost, 
    COALESCE(tres.total_cost_lost, 0) AS total_cost_lost,
    COALESCE(quatro.total_cost, 0) AS total_cost, 
    COALESCE(cinco.total_salary, 0) AS total_salary
FROM {_schema}.lots
    LEFT JOIN cero ON lots.lot_id = cero.lot_id
	LEFT JOIN uno ON lots.lot_id = uno.lot_id
	LEFT JOIN due ON lots.lot_id = due.lot_id
	LEFT JOIN tres ON lots.lot_id = tres.lot_id
    LEFT JOIN quatro ON lots.lot_id = quatro.lot_id
    LEFT JOIN cinco ON lots.lot_id = cinco.lot_id
ORDER BY lots.lot_id DESC;

COMMENT ON VIEW {_schema}.{name}
    IS '{comment}';
"""
# create view
print(crt_query(stat))

## VIEW  lot_model_totals

In [ ]:
name = 'total_lot_model_per_lot'
comment = 'Totals from lot_models per lot'

stat = f"""
CREATE OR REPLACE VIEW {_schema}.{name}
AS
SELECT lm.lot_id,
    CASE WHEN BOOL_OR(lm.weight IS NULL) THEN NULL 
        ELSE SUM(lm.weight) END AS total_weight_models,
	COUNT(lm.model_id ) AS model_count,
	SUM( c.pieces ) AS pieces_sum
FROM {_schema}.lot_models AS lm
 LEFT JOIN {_schema}.catalog AS c ON lm.model_id = c.model_id
GROUP BY lm.lot_id
ORDER BY lm.lot_id DESC;

COMMENT ON VIEW {_schema}.{name}
    IS '{comment}';
"""
# create view
print(crt_query(stat))

## VIEW salaries_per_lot

In [ ]:
name = 'salaries_per_lot'
comment = 'Calculates salaries per lot per type'

stat = f"""
CREATE OR REPLACE VIEW {_schema}.{name}
AS
WITH uno AS(
    SELECT lot_id,
        SUM(cost_manufacturing_stone * quantity::numeric) as cost_manufacturing_stone
    FROM {_schema}.lot_model_stones
    GROUP BY lot_id
)
SELECT lots.lot_id, 
    tcpl.total_salary AS cost_casting, 
    lots.cost_grinding * tppl.total_weight_in AS cost_grinding,
    uno.cost_manufacturing_stone,
    lots.cost_polishing * tlmpl.pieces_sum AS cost_polishing,
    lots.cost_plating * tlmpl.pieces_sum AS cost_plating,
    lots.cost_sinji * tlmpl.pieces_sum AS cost_sinji, 
    ROUND(lots.cost_grinding * tppl.total_weight_in +
        uno.cost_manufacturing_stone + lots.cost_polishing * tlmpl.pieces_sum +
        lots.cost_plating * tlmpl.pieces_sum + 
        lots.cost_sinji * tlmpl.pieces_sum, 2) AS total_salary
FROM {_schema}.lots
	LEFT JOIN {_schema}.total_cast_per_lot AS tcpl ON lots.lot_id = tcpl.lot_id
	LEFT JOIN {_schema}.total_processing_per_lot AS tppl ON lots.lot_id = tppl.lot_id
	LEFT JOIN {_schema}.total_lot_model_per_lot AS tlmpl ON lots.lot_id = tlmpl.lot_id
    LEFT JOIN uno ON lots.lot_id = uno.lot_id
ORDER BY lots.lot_id DESC;

COMMENT ON VIEW {_schema}.{name}
    IS '{comment}';
"""
# create view
print(crt_query(stat))

## VIEW total_all_stones_per_lot

In [ ]:
name = 'total_all_stones_per_lot'
comment = 'total all stones used per lot despite weighted or unweighted'

stat = f"""
CREATE OR REPLACE VIEW {_schema}.{name}
AS
SELECT lms.lot_id, lms.stone_full_name,
	SUM(lms.quantity) AS quantity, 
	SUM(lms.total_weight) AS total_weight, lms.weight_unit, 
    CASE WHEN BOOL_OR(lms.cost_piece IS NULL) THEN NULL 
        ELSE SUM(lms.total_cost_piece) END AS total_cost_piece,
    CASE WHEN BOOL_OR(lms.cost_manufacturing_stone IS NULL) THEN NULL 
        ELSE SUM(lms.total_cost_manufacturing_stone) END AS total_cost_manufacturing_stone,
	CASE WHEN BOOL_OR(lms.margin_stones IS NULL) THEN NULL 
        ELSE SUM(lms.total_margin_stones) END AS total_margin_stones,
    SUM(lms.total_cost) AS total_cost,
	SUM(lms.total_price) AS total_price
FROM {_schema}.lot_model_stones AS lms 
	LEFT JOIN {_schema}.lot_models AS lm 
		ON lms.lot_id = lm.lot_id
		AND lms.model_id = lm.model_id
		AND lms.tmstmp = lm.tmstmp
GROUP BY lms.lot_id, lms.stone_full_name, lms.weight_unit;

COMMENT ON VIEW {_schema}.{name}
    IS '{comment}';
"""
# create view
print(crt_query(stat))

## VIEW cost_calculation_per_lot

In [ ]:
name = 'cost_calculation_per_lot'
comment = 'Calculates cost per gram for each lot, doesnt include stone costs'

stat = f"""
CREATE OR REPLACE VIEW {_schema}.{name}
AS
WITH uno AS(
    SELECT lot_id,
        SUM(total_cost) AS total_stone_cost
    FROM total_all_stones_per_lot
    GROUP BY lot_id
)
SELECT lots.lot_id, 
    tppl.total_weight_out, tppl.total_cost_out,
    tcpl.total_weight_lost + tppl.total_weight_lost AS total_weight_lost,
    tcpl.total_cost_lost + tppl.total_cost_lost AS total_cost_lost,
    tcpl.total_cost + tppl.total_cost AS total_salary_other_cost,
    ROUND(tcpl.total_cost_lost + tcpl.total_cost + 
        tppl.total_cost_out + tppl.total_cost_lost +
        spl.total_salary - spl.cost_manufacturing_stone, 2) AS total_cost,
	ROUND((tcpl.total_cost_lost + tcpl.total_cost + 
        tppl.total_cost_out + tppl.total_cost_lost +
        spl.total_salary - spl.cost_manufacturing_stone) / 
        tppl.total_weight_out, 2) AS cost_per_gram,
    ROUND(tcpl.total_cost_lost + tcpl.total_cost + 
        tppl.total_cost_out + tppl.total_cost_lost +
        spl.total_salary - spl.cost_manufacturing_stone +
        total_stone_cost, 2) AS total_lot_cost   
FROM {_schema}.lots
	LEFT JOIN {_schema}.total_cast_per_lot AS tcpl ON lots.lot_id = tcpl.lot_id
	LEFT JOIN {_schema}.total_processing_per_lot AS tppl ON lots.lot_id = tppl.lot_id
	LEFT JOIN {_schema}.total_lot_model_per_lot AS tlmpl ON lots.lot_id = tlmpl.lot_id
    LEFT JOIN {_schema}.salaries_per_lot AS spl ON lots.lot_id = spl.lot_id
    LEFT JOIN uno ON lots.lot_id = uno.lot_id
ORDER BY lots.lot_id DESC;

COMMENT ON VIEW {_schema}.{name}
    IS '{comment}';
"""
# create view
print(crt_query(stat))

## VIEW total_stones_stock

In [ ]:
name = 'total_stones_stock'
comment = 'total stones in stock'

stat = f"""
CREATE OR REPLACE VIEW {_schema}.{name}
AS
SELECT item AS stone_full_name,
	SUM(transaction_quantity) AS transaction_quantity, transaction_quantity_unit,
	SUM(pieces) AS pieces,
	SUM(total_cost) AS total_price,
	(SUM(total_cost) / SUM(pieces))::numeric(10, 2) AS average_price_piece
FROM {_schema}.transactions
WHERE item_type = 'ქვა'
GROUP BY item, transaction_quantity_unit
ORDER BY item;

COMMENT ON VIEW {_schema}.{name}
    IS '{comment}';
"""
# create view
print(crt_query(stat))

## VIEW total_uninstalled_stones_per_lot

In [ ]:
name = 'total_uninstalled_stones_per_lot'
comment = 'total uninstalled stones used per lot'

stat = f"""
CREATE OR REPLACE VIEW {_schema}.{name}
AS
SELECT lot_id, stone_full_name,
	SUM(quantity) AS quantity, 
	SUM(total_weight) AS total_weight, weight_unit, 
    CASE WHEN BOOL_OR(cost_piece IS NULL) THEN NULL 
        ELSE SUM(total_cost_piece) END AS total_cost_piece,
    CASE WHEN BOOL_OR(cost_manufacturing_stone IS NULL) THEN NULL 
        ELSE SUM(total_cost_manufacturing_stone) END AS total_cost_manufacturing_stone,
	CASE WHEN BOOL_OR(margin_stones IS NULL) THEN NULL 
        ELSE SUM(total_margin_stones) END AS total_margin_stones,
    SUM(total_cost) AS total_cost,
	SUM(total_price) AS total_price
FROM {_schema}.lot_model_stones
WHERE installed = false
GROUP BY lot_id, stone_full_name, weight_unit;

COMMENT ON VIEW {_schema}.{name}
    IS '{comment}';
"""
# create view
print(crt_query(stat))

## VIEW lot_stone_totals

In [ ]:
name = 'lot_stone_totals'
comment = 'specific, stones summary table per lot'

stat = f"""
CREATE OR REPLACE VIEW {_schema}.{name}
AS
SELECT 
	CASE 
        WHEN taspl.weight_unit <> tss.transaction_quantity_unit 
            THEN 'Err'
        WHEN taspl.total_cost_piece is NULL
            THEN 'Err'      
        WHEN taspl.total_cost_manufacturing_stone is NULL
            THEN 'Err'    
        WHEN taspl.total_margin_stones is NULL
            THEN 'Err'                
        ELSE '' 
    END AS err,
	taspl.lot_id, taspl.stone_full_name,
    taspl.quantity::integer, tss.pieces::integer, 
    taspl.total_weight, taspl.weight_unit, tss.transaction_quantity, tss.transaction_quantity_unit,
    taspl.total_cost_piece,  ROUND(taspl.total_cost_piece / taspl.quantity, 2) AS avg_cost_piece,
    taspl.total_cost_manufacturing_stone, taspl.total_cost, 
	taspl.total_margin_stones, taspl.total_price
--	tss.total_price, tss.average_price_piece
FROM {_schema}.total_all_stones_per_lot AS taspl
	LEFT JOIN {_schema}.total_stones_stock AS tss ON taspl.stone_full_name = tss.stone_full_name;

COMMENT ON VIEW {_schema}.{name}
    IS '{comment}';
"""
# create view
print(crt_query(stat))

## VIEW lot_stone_validation_totals

In [ ]:
name = 'lot_stone_validation_totals'
comment = 'total stones per lot that are needed for manufacturing'

stat = f"""
CREATE OR REPLACE VIEW {_schema}.{name}
AS
SELECT 
	CASE 
        WHEN tuspl.weight_unit <> tss.transaction_quantity_unit 
				OR tuspl.weight_unit IS NULL 
				OR tss.transaction_quantity_unit  IS NULL
            THEN 'Err'
        WHEN tss.pieces - tuspl.quantity < 0
            THEN 'Err' 
        WHEN tuspl.total_cost_piece is NULL
            THEN 'Err'        
        WHEN tuspl.total_cost_manufacturing_stone is NULL
            THEN 'Err'       
        WHEN tuspl.total_margin_stones is NULL
            THEN 'Err'                   
        ELSE '' 
    END AS err,
	tuspl.lot_id, tuspl.stone_full_name,
    tuspl.quantity::integer, tss.pieces::integer, 
    tuspl.total_weight, tuspl.weight_unit, tss.transaction_quantity, tss.transaction_quantity_unit,
    tuspl.total_cost_piece,  ROUND(tuspl.total_cost_piece / tuspl.quantity, 2) AS avg_cost_piece,
    tuspl.total_cost_manufacturing_stone, tuspl.total_cost, 
	tuspl.total_margin_stones, tuspl.total_price
--	tss.total_price, tss.average_price_piece
FROM {_schema}.total_uninstalled_stones_per_lot AS tuspl
	LEFT JOIN {_schema}.total_stones_stock AS tss ON tuspl.stone_full_name = tss.stone_full_name;

COMMENT ON VIEW {_schema}.{name}
    IS '{comment}';
"""
# create view
print(crt_query(stat))

## VIEW lot_update_models

In [ ]:
name = 'lot_update_models'
comment = 'view for django view lot_update variable lot_models'

stat = f"""
CREATE OR REPLACE VIEW {_schema}.{name}
AS
SELECT c.image_location, lm.lot_id, lm.model_id, lm.tmstmp, lm.location, lm.customer, 
	lm.weight, lm.cost_gram_gold, lm.price_gram_gold,
	SUM(lms.total_weight) AS model_total_stone_weight,
	SUM(lms.quantity) AS model_total_stone_quantity,
    CASE WHEN BOOL_OR(lms.total_cost_piece IS NULL) 
        THEN NULL 
        ELSE SUM(lms.total_cost_piece) 
    END AS model_total_stone_cost_piece,
    CASE WHEN BOOL_OR(lms.total_cost_manufacturing_stone IS NULL) 
        THEN NULL 
        ELSE SUM(lms.total_cost_manufacturing_stone) 
    END AS model_total_stone_cost_manufacturing_stone,
    CASE WHEN BOOL_OR(lms.total_margin_stones IS NULL) 
        THEN NULL 
        ELSE SUM(lms.total_margin_stones)
    END AS model_total_stone_margin_stones,
    CASE WHEN SUM(lms.quantity) is NULL THEN lm.model_gold_cost
        WHEN BOOL_OR(lms.total_cost IS NULL) THEN NULL
        ELSE SUM(lms.total_cost) + lm.model_gold_cost 
    END AS model_cost,
    CASE WHEN SUM(lms.quantity) is NULL THEN lm.model_gold_price
        WHEN BOOL_OR(lms.total_price IS NULL) THEN NULL
        ELSE SUM(lms.total_price) + lm.model_gold_price 
    END model_price
FROM {_schema}.lot_models AS lm
    LEFT JOIN {_schema}.lot_model_stones AS lms ON lm.tmstmp = lms.tmstmp
    LEFT JOIN {_schema}.catalog AS c ON lm.model_id = c.model_id
GROUP BY c.image_location, lm.lot_id, lm.model_id, lm.tmstmp, lm.location, 
	lm.weight, lm.cost_gram_gold, lm.price_gram_gold
ORDER BY lm.location DESC, lm.lot_id DESC, lm.model_id, lm.weight, lm.tmstmp DESC;

COMMENT ON VIEW {_schema}.{name}
    IS '{comment}';
"""
# create view
print(crt_query(stat))

## VIEW lot_update_stones

In [ ]:
name = 'lot_update_stones'
comment = 'view for django view lot_update variable lot_stones'

stat = f"""
CREATE OR REPLACE VIEW {_schema}.{name}
AS
SELECT tmstmp, stone_full_name, total_weight, quantity, 
    total_cost_piece, total_cost_manufacturing_stone, total_margin_stones  
FROM {_schema}.lot_model_stones 
ORDER BY stone_full_name;

COMMENT ON VIEW {_schema}.{name}
    IS '{comment}';
"""
# create view
print(crt_query(stat))

## VIEW customer_details

In [ ]:
name = 'customer_details'
comment = 'customer purchase details includintg remaining payments'

stat = f"""
CREATE OR REPLACE VIEW {_schema}.{name}
AS
WITH uno AS(
	SELECT full_name, phone,
		COALESCE(SUM(total_cost), 0) AS paid
	FROM {_schema}.customers AS c
		LEFT JOIN {_schema}.transactions AS t ON c.full_name = t.customer
	GROUP BY full_name, phone
), due AS(
SELECT c.full_name AS due_full_name, lum.location,
	COUNT(DISTINCT lum.lot_id) AS distinct_lots_count,
	COUNT(lum.model_id) AS models_count, 
	COUNT(DISTINCT lum.model_id) AS distinct_models_count,
	SUM(lum.weight) AS total_model_weight, 
	SUM(lum.model_total_stone_weight) AS total_stone_weight, 
	SUM(lum.model_total_stone_quantity) AS total_stone_quantity,
	SUM(lum.model_cost) AS total_cost,
	SUM(lum.model_price) AS total_price,
	SUM(lum.model_price) - SUM(lum.model_cost) AS total_profit
FROM {_schema}.customers AS c
	LEFT JOIN {_schema}.lot_update_models AS lum ON c.full_name = lum.customer
GROUP BY full_name, lum.location
ORDER BY full_name, lum.location
)
SELECT *, paid - total_price AS remaining_payment
FROM uno
	LEFT JOIN due ON full_name = due_full_name;
--WHERE location = 'გაყიდული';

COMMENT ON VIEW {_schema}.{name}
    IS '{comment}';
"""
# create view
print(crt_query(stat))